# CkipTagger 中文斷詞工具


參考網址：https://colab.research.google.com/drive/1ZnB46vPPiClDxEulZ27BD1WEsNohFYey#scrollTo=5OcQ66GuSL3I


## 安裝套件




In [1]:
!pip install -U ckiptagger[tf,gdown]

In [2]:
!pip install -U ckiptagger

## 使用 CKIP



In [3]:
from ckiptagger import data_utils, construct_dictionary, WS, POS, NER

### 1. 下載模型




In [4]:
# Downloads to ./data.zip (2GB) and extracts to ./data/
data_utils.download_data_url("./") # iis-ckip
# data_utils.download_data_gdown("./") # gdrive-ckip

- ./data/model_ner/pos_list.txt -> POS tag list, see [Wiki](https://github.com/ckiplab/ckiptagger/wiki/POS-Tags) / [Technical Report no. 93-05](http://ckip.iis.sinica.edu.tw/CKIP/tr/9305_2013%20revision.pdf)
- ./data/model_ner/label_list.txt -> Entity type list, see [Wiki](https://github.com/ckiplab/ckiptagger/wiki/Entity-Types) / [OntoNotes Release 5.0](https://catalog.ldc.upenn.edu/docs/LDC2013T19/OntoNotes-Release-5.0.pdf)
- ./data/embedding_* -> character/word embeddings, see [Wiki](https://github.com/ckiplab/ckiptagger/wiki/Corpora)


### 2. 載入模型


In [5]:
# To use GPU:
#    1. Install tensorflow-gpu (see Installation)
#    2. Set CUDA_VISIBLE_DEVICES environment variable, e.g. os.environ["CUDA_VISIBLE_DEVICES"] = "0"
#    3. Set disable_cuda=False, e.g. ws = WS("./data", disable_cuda=False)
# To use CPU:
ws = WS("./data") # 斷詞
pos = POS("./data") # 詞性標註
ner = NER("./data") # 命名實體辨識

/usr/local/lib/python3.10/dist-packages/ckiptagger/model_ws.py:106: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  cell = tf.compat.v1.nn.rnn_cell.LSTMCell(hidden_d, name=name)
/usr/local/lib/python3.10/dist-packages/ckiptagger/model_pos.py:56: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  cell = tf.compat.v1.nn.rnn_cell.LSTMCell(hidden_d, name=name)
/usr/local/lib/python3.10/dist-packages/ckiptagger/model_ner.py:57: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  cell = tf.compat.v1.nn.rnn_cell.LSTMCell(hidden_d, name=name)


### 3. 執行 WS(斷詞)-POS(詞性標註)-NER(命名實體辨識) pipeline

In [6]:
sentence_list = [
    "傅達仁今將執行安樂死，卻突然爆出自己20年前遭緯來體育台封殺，他不懂自己哪裡得罪到電視台。",
    "美國參議院針對今天總統布什所提名的勞工部長趙小蘭展開認可聽證會，預料她將會很順利通過參議院支持，成為該國有史以來第一位的華裔女性內閣成員。",
    "",
    "土地公有政策?？還是土地婆有政策。.",
    "… 你確定嗎… 不要再騙了……",
    "最多容納59,000個人,或5.9萬人,再多就不行了.這是環評的結論.",
    "科長說:1,坪數對人數為1:3。2,可以再增加。",
    "其中，台灣首府大學於2000年在台南麻豆建校，前身為致遠管理學院",
    "全盛時期學生人數近1萬人，設有教育與創新管理學院、餐旅管理學院",
    "但招生人數銳減，2021年僅47名新生註冊，註冊率23.3%。",
    "台灣首府大學為了撐下去，雖一度改為獨招，並且積極招收陸生、外籍生來改善狀況",
    "但財部困難嚴重，董事會因無法在期限內籌足資金，於去年8月1日起停辦。",
]


In [7]:
word_sentence_list = ws(
    sentence_list,
    # sentence_segmentation = True, # To consider delimiters
    # segment_delimiter_set = {",", "。", ":", "?", "!", ";"}), # This is the defualt set of delimiters
    # recommend_dictionary = dictionary1, # words in this dictionary are encouraged
    # coerce_dictionary = dictionary2, # words in this dictionary are forced
)

In [8]:
print(word_sentence_list)

[['傅達仁', '今', '將', '執行', '安樂死', '，', '卻', '突然', '爆出', '自己', '20', '年', '前', '遭', '緯來', '體育台', '封殺', '，', '他', '不', '懂', '自己', '哪裡', '得罪到', '電視台', '。'], ['美國', '參議院', '針對', '今天', '總統', '布什', '所', '提名', '的', '勞工部長', '趙小蘭', '展開', '認可', '聽證會', '，', '預料', '她', '將', '會', '很', '順利', '通過', '參議院', '支持', '，', '成為', '該', '國', '有史以來', '第一', '位', '的', '華裔', '女性', '內閣', '成員', '。'], [], ['土地公', '有', '政策', '?', '？', '還是', '土地', '婆', '有', '政策', '。', '.'], ['…', ' ', '你', '確定', '嗎', '…', ' ', '不要', '再', '騙', '了', '…', '…'], ['最多', '容納', '59,000', '個', '人', ',', '或', '5.9萬', '人', ',', '再', '多', '就', '不行', '了', '.', '這', '是', '環評', '的', '結論', '.'], ['科長', '說', ':1,', '坪數', '對', '人數', '為', '1:3', '。', '2', ',', '可以', '再', '增加', '。'], ['其中', '，', '台灣', '首府', '大學', '於', '2000年', '在', '台南', '麻豆', '建', '校', '，', '前身', '為', '致遠', '管理', '學院'], ['全盛', '時期', '學生', '人數', '近', '1萬', '人', '，', '設有', '教育', '與', '創新', '管理', '學院', '、', '餐旅', '管理', '學院'], ['但', '招生', '人數', '銳減', '，', '2021年', '僅', '47', '名', '新生', '註冊', 

In [9]:
pos_sentence_list = pos(word_sentence_list)

參考網址：https://github.com/ckiplab/ckiptagger/wiki/POS-Tags

In [10]:
print(pos_sentence_list)

[['Nb', 'Nd', 'D', 'VC', 'Na', 'COMMACATEGORY', 'D', 'D', 'VJ', 'Nh', 'Neu', 'Nf', 'Ng', 'P', 'Nb', 'Na', 'VC', 'COMMACATEGORY', 'Nh', 'D', 'VK', 'Nh', 'Ncd', 'VJ', 'Nc', 'PERIODCATEGORY'], ['Nc', 'Nc', 'P', 'Nd', 'Na', 'Nb', 'D', 'VC', 'DE', 'Na', 'Nb', 'VC', 'VC', 'Na', 'COMMACATEGORY', 'VE', 'Nh', 'D', 'D', 'Dfa', 'VH', 'VC', 'Nc', 'VC', 'COMMACATEGORY', 'VG', 'Nes', 'Nc', 'D', 'Neu', 'Nf', 'DE', 'Na', 'Na', 'Na', 'Na', 'PERIODCATEGORY'], [], ['Nb', 'V_2', 'Na', 'QUESTIONCATEGORY', 'QUESTIONCATEGORY', 'Caa', 'Na', 'Na', 'V_2', 'Na', 'PERIODCATEGORY', 'PERIODCATEGORY'], ['ETCCATEGORY', 'WHITESPACE', 'Nh', 'VK', 'T', 'ETCCATEGORY', 'WHITESPACE', 'D', 'D', 'VC', 'Di', 'ETCCATEGORY', 'ETCCATEGORY'], ['VH', 'VJ', 'Neu', 'Nf', 'Na', 'COMMACATEGORY', 'Caa', 'Neu', 'Na', 'COMMACATEGORY', 'D', 'D', 'D', 'VH', 'T', 'PERIODCATEGORY', 'Nep', 'SHI', 'Na', 'DE', 'Na', 'PERIODCATEGORY'], ['Na', 'VE', 'Neu', 'Na', 'P', 'Na', 'VG', 'Neu', 'PERIODCATEGORY', 'Neu', 'COMMACATEGORY', 'D', 'D', 'VHC', 'P

In [11]:
entity_sentence_list = ner(word_sentence_list, pos_sentence_list)

In [12]:
print(entity_sentence_list)

[{(23, 28, 'ORG', '緯來體育台'), (0, 3, 'PERSON', '傅達仁'), (18, 22, 'DATE', '20年前')}, {(17, 21, 'ORG', '勞工部長'), (60, 62, 'NORP', '華裔'), (21, 24, 'PERSON', '趙小蘭'), (42, 45, 'ORG', '參議院'), (0, 2, 'GPE', '美國'), (11, 13, 'PERSON', '布什'), (7, 9, 'DATE', '今天'), (56, 58, 'ORDINAL', '第一'), (2, 5, 'ORG', '參議院')}, set(), {(0, 3, 'PERSON', '土地公')}, set(), {(14, 18, 'CARDINAL', '5.9萬'), (4, 10, 'CARDINAL', '59,000')}, {(16, 17, 'CARDINAL', '2'), (4, 6, 'CARDINAL', '1,'), (14, 15, 'CARDINAL', '3'), (12, 13, 'CARDINAL', '1')}, {(5, 9, 'ORG', '首府大學'), (16, 20, 'GPE', '台南麻豆'), (3, 5, 'GPE', '台灣'), (26, 32, 'ORG', '致遠管理學院'), (10, 15, 'DATE', '2000年')}, {(8, 11, 'CARDINAL', '近1萬')}, {(14, 16, 'CARDINAL', '47'), (8, 13, 'DATE', '2021年'), (25, 30, 'PERCENT', '23.3%')}, {(0, 6, 'ORG', '台灣首府大學')}, {(1, 3, 'ORG', '財部'), (24, 30, 'DATE', '去年8月1日')}]


### 4. 顯示結果


In [13]:
def print_word_pos_sentence(word_sentence, pos_sentence):
    assert len(word_sentence) == len(pos_sentence)
    for word, pos in zip(word_sentence, pos_sentence):
        print(f"{word}({pos})", end="\u3000")
    print()
    return

In [14]:
for i, sentence in enumerate(sentence_list):
    print()
    print(f"'{sentence}'")
    print_word_pos_sentence(word_sentence_list[i],  pos_sentence_list[i])
    for entity in sorted(entity_sentence_list[i]):
        print(entity)


'傅達仁今將執行安樂死，卻突然爆出自己20年前遭緯來體育台封殺，他不懂自己哪裡得罪到電視台。'
傅達仁(Nb)　今(Nd)　將(D)　執行(VC)　安樂死(Na)　，(COMMACATEGORY)　卻(D)　突然(D)　爆出(VJ)　自己(Nh)　20(Neu)　年(Nf)　前(Ng)　遭(P)　緯來(Nb)　體育台(Na)　封殺(VC)　，(COMMACATEGORY)　他(Nh)　不(D)　懂(VK)　自己(Nh)　哪裡(Ncd)　得罪到(VJ)　電視台(Nc)　。(PERIODCATEGORY)　
(0, 3, 'PERSON', '傅達仁')
(18, 22, 'DATE', '20年前')
(23, 28, 'ORG', '緯來體育台')

'美國參議院針對今天總統布什所提名的勞工部長趙小蘭展開認可聽證會，預料她將會很順利通過參議院支持，成為該國有史以來第一位的華裔女性內閣成員。'
美國(Nc)　參議院(Nc)　針對(P)　今天(Nd)　總統(Na)　布什(Nb)　所(D)　提名(VC)　的(DE)　勞工部長(Na)　趙小蘭(Nb)　展開(VC)　認可(VC)　聽證會(Na)　，(COMMACATEGORY)　預料(VE)　她(Nh)　將(D)　會(D)　很(Dfa)　順利(VH)　通過(VC)　參議院(Nc)　支持(VC)　，(COMMACATEGORY)　成為(VG)　該(Nes)　國(Nc)　有史以來(D)　第一(Neu)　位(Nf)　的(DE)　華裔(Na)　女性(Na)　內閣(Na)　成員(Na)　。(PERIODCATEGORY)　
(0, 2, 'GPE', '美國')
(2, 5, 'ORG', '參議院')
(7, 9, 'DATE', '今天')
(11, 13, 'PERSON', '布什')
(17, 21, 'ORG', '勞工部長')
(21, 24, 'PERSON', '趙小蘭')
(42, 45, 'ORG', '參議院')
(56, 58, 'ORDINAL', '第一')
(60, 62, 'NORP', '華裔')

''


'土地公有政策?？還是土地婆有政策。.'
土地公(Nb)　有(V_2)　政策(Na)　?(QUESTIONCATEGORY)　？(QUESTIONCATEGORY)　還是(Caa)　土地(N

## 請使用CKIP套件將從淡江學務處爬下來的5個網頁進行NER關鍵字提取，請列出5個網址及取出之關鍵字。



*   淡江：https://www.tku.edu.tw/regis.asp
*   淡江時報：https://tkutimes.tku.edu.tw/index.cshtml
*   教務處註冊組：https://atgx.acad.tku.edu.tw/
*   教務處課務組：https://atcx.acad.tku.edu.tw/

*   學生事務處：http://spirit.tku.edu.tw/tku/home.jsp
*   諮商職涯暨學習發展輔導中心：http://spirit.tku.edu.tw/tku/main.jsp?sectionId=4
*   原住民族學生資源中心：http://spirit.tku.edu.tw/tku/main.jsp?sectionId=9
*   衛生保健組：http://spirit.tku.edu.tw/tku/main.jsp?sectionId=6
*   生活輔導組：http://spirit.tku.edu.tw/tku/main.jsp?sectionId=2
*   課外活動輔導組：http://spirit.tku.edu.tw/tku/main.jsp?sectionId=3
*   住宿輔導組：http://spirit.tku.edu.tw/tku/main.jsp?sectionId=8

In [1]:
!pip install -U ckiptagger[tf,gdown]

In [2]:
!pip install -U ckiptagger

In [4]:
import requests
from bs4 import BeautifulSoup
from ckiptagger import data_utils, construct_dictionary, WS, POS, NER
import numpy as np

In [5]:
# Downloads to ./data.zip (2GB) and extracts to ./data/
data_utils.download_data_url("./") # iis-ckip
# data_utils.download_data_gdown("./") # gdrive-ckip

# To use CPU:
ws = WS("./data") # 斷詞
pos = POS("./data") # 詞性標註
ner = NER("./data") # 命名實體辨識

/usr/local/lib/python3.10/dist-packages/ckiptagger/model_ws.py:106: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  cell = tf.compat.v1.nn.rnn_cell.LSTMCell(hidden_d, name=name)
/usr/local/lib/python3.10/dist-packages/ckiptagger/model_pos.py:56: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  cell = tf.compat.v1.nn.rnn_cell.LSTMCell(hidden_d, name=name)
/usr/local/lib/python3.10/dist-packages/ckiptagger/model_ner.py:57: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  cell = tf.compat.v1.nn.rnn_cell.LSTMCell(hidden_d, name=name)


In [9]:
def content(url):
    print("網址:", url)

    # 抓取網頁內容
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    sentence_list = soup.get_text()

    # 進行斷詞、詞性標注、實體辨識
    word_sentence_list = ws([sentence_list])
    pos_sentence_list = pos(word_sentence_list)
    entity_sentence_list = ner(word_sentence_list, pos_sentence_list)

    # 提取非特定實體類別的關鍵字
    keywords = []
    for entity in entity_sentence_list[0]:
        if entity[2] != 'PRODUCT' and entity[2] != 'DATE' and entity[2] != 'CARDINAL' and entity[2] != 'ORDINAL':
            keywords.append(entity[3])

    print("關鍵字：", keywords)
    print()


# 學生事務處各組的網址
urls = ["http://spirit.tku.edu.tw/tku/main.jsp?sectionId=1", # 學務處長室
      "http://spirit.tku.edu.tw/tku/main.jsp?sectionId=2", # 生活輔導組
      "http://spirit.tku.edu.tw/tku/main.jsp?sectionId=3", # 課外活動輔導組
      "http://spirit.tku.edu.tw/tku/main.jsp?sectionId=4", # 諮商職涯暨學習發展輔導中心
      "http://spirit.tku.edu.tw/tku/main.jsp?sectionId=6", # 衛生保衛組
      "http://spirit.tku.edu.tw/tku/main.jsp?sectionId=8", # 住宿輔導組
      "http://spirit.tku.edu.tw/tku/main.jsp?sectionId=9", # 原住民族學生資源中心
      ]

# 依次提取每個網址的內容並進行處理
for url in urls:
    content(url)

網址: http://spirit.tku.edu.tw/tku/main.jsp?sectionId=1
關鍵字： ['新北市', '淡江大學學生事務處']

網址: http://spirit.tku.edu.tw/tku/main.jsp?sectionId=2
關鍵字： ['English version', 'Rewards and Disciplines on Students\xa0', 'Tutoring', 'Transportation Safety校園安全Campus Security防制校園', 'Military Service Illustration', '淡江大學學生事務處', '役政司', 'Subsidies for Aborigines', '王紹新', '董俊章', '內政部', '國防部', '中文版', 'Foundation', '新北市', 'Subsidies for Graduate Students生活教育券補助金碩博士班', 'Outstanding Youth Awards', 'Student Appeal 防制學生藥物濫用資源網Echinacea', '張國基', '淡大-公館', 'Counseling\xa0']

網址: http://spirit.tku.edu.tw/tku/main.jsp?sectionId=3
關鍵字： ['中文版', '新北市', '中文版', 'School Club Leaders\xa0課外組', '英文版', '中文版', '英文版', '中文版', '淡江大學學生事務處', 'Hall of Fame\xa0', '英文版', 'Information about club activity funding\xa0', 'College-Assisted Club Development Proj. of High & Elementary Schools\xa0教育部', '英文版', 'List of Partner Institutions社團', '中文版', '英文版', '中文版', '英文版']

網址: http://spirit.tku.edu.tw/tku/main.jsp?sectionId=4
關鍵字： ['Study', '教育部', 

#### DeBug中的辦法


In [ ]:
# 正則表達式：http://spirit\.tku\.edu\.tw/tku/.+\.jsp.*

In [ ]:
!pip install scrapy jieba pandas ckiptagger

In [ ]:
!scrapy startproject keyword_crawler
!cd keyword_crawler

In [ ]:
!scrapy runspider keyword_spider.py -o keyword_spider.jl

In [3]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""Created on Fri Mar  8 01:59:40 2024 @author: 410770605-洪翊婕"""

'Created on Fri Mar  8 01:59:40 2024\n\n@author: 410770605-洪翊婕'